In [1]:
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, TextBox, RadioButtons
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from BPS.synthesis import prior_pdf, synthesise, synthesise_2_agents
from BPS.synthesis_plots import plot, plot_2_agents
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
def update(val):
    global sigma 
    global mu
    global f
    global s
    sigma = sig_slider.val
    mu = mu_slider.val
    f = f_slider.val
    s = s_slider.val
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    axs.lines[3].set_ydata(st.norm(loc=f, scale=s).pdf(xs))
    fig.canvas.draw_idle()

# Bayesian Predictive Synthesis
Based on ``Bayesian Predictive Synthesis: Forecast Calibration and Combination'' (2018) by Matthew C. Johnson and Mike West. All mistakes (if any are present) are mine and not the authors of the paper.

## Gaussian kernel, 1 agent

$$m(x) \sim N(0, 1) $$
$$\pi_0(x) \sim N(0,1) $$

Agent's forecasts are $h(x)$, and $m(x)=E[h(x)]$ that is it's your expectation as to what agent forecast will be.

The user can select one of the three possible synthesis functions:
$$a(x) = exp[-(x-\mu)^2 / (2 r_1 \sigma^2)] $$
$$a(x) = 1 - dexp[-(x-\mu)^2 / (2 r_2 \sigma^2)] $$
$$a(x) =  exp[-(x-\mu)^2 / (2 r_1 \sigma^2)]\left(1 - dexp[-(x-\mu)^2 / (2 r_2 \sigma^2)]\right) $$

In [8]:
q = 0.5
r_1 = 1.0
r_2 = 1.0
mu = 0
sigma = 1
d = 1
pi_0 = st.norm(loc=0, scale=1)
mx = st.norm(loc=0, scale=1)# *np.sqrt(r)
ax = lambda x: np.exp(-(x-mu)**2 / (2 * r_1 * sigma**2))
ax_dict = {
            "$a(x) = exp[-(x-\mu)^2 / (2 r \sigma^2)]$": lambda x: np.exp(-(x-mu)**2 / (2 * r_1 * sigma**2)),
            "$a(x) = 1 - dexp[-(x-\mu)^2 / (2 r \sigma^2)]$": lambda x: 1 - d * np.exp(-(x-mu)**2 / (2*r_2*sigma**2)),
            "$a(x) = exp[-(x-\mu)^2 / (2 r \sigma^2)](1 - dexp[-(x-\mu)^2 / (2 r \sigma^2)])$": lambda x: np.exp(-(x-mu)**2 / (2 * r_1 * sigma**2)) * (1 - d * np.exp(-(x-mu)**2 / (2*r_2*sigma**2)))
            }

In [10]:
fig, axs = plt.subplots(1,1, figsize=(10,7.5))
f = 1; s = np.sqrt(0.1); xs = np.linspace(-5, 5, 1000)
plot(axs, xs, q, ax, mx, pi_0, f, s)
plt.subplots_adjust(left=0.25, bottom=0.25, top=0.85)

axf = plt.axes([0.25, 0.14, 0.65, 0.02], facecolor='white')
axstd = plt.axes([0.25, 0.17, 0.65, 0.02], facecolor='white')
ax_mu = plt.axes([0.25, 0.11, 0.65, 0.02], facecolor='white')
ax_sig = plt.axes([0.25, 0.08, 0.65, 0.02], facecolor='white')
radio_ax = plt.axes([0.25, 0.85, 0.65, 0.15], facecolor='white')
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])

button = Button(resetax, 'Reset', color="white", hovercolor='0.975')
def reset(event):
    f_slider.reset()
    s_slider.reset()
    mu_slider.reset()
    f_slider.reset()
    
button.on_clicked(reset)

q_ax = fig.add_axes([0.1, 0.8, 0.05, 0.05])# plt.axes([0.1, 0.25, 0.0225, 0.63], facecolor=axcolor)
q_box = TextBox(q_ax, "$q$:", initial="0.5")

r_ax = fig.add_axes([0.1, 0.7, 0.05, 0.05])
r_box = TextBox(r_ax, "$r_1$:", initial="1.0")

r2_ax = fig.add_axes([0.1, 0.6, 0.05, 0.05])
r2_box = TextBox(r2_ax, "$r_2$:", initial="1.0")

d_ax = fig.add_axes([0.1, 0.5, 0.05, 0.05])
d_box = TextBox(d_ax, "$d$:", initial="1.0")

radio = RadioButtons(radio_ax, ('$a(x) = exp[-(x-\mu)^2 / (2 r \sigma^2)]$', '$a(x) = 1 - dexp[-(x-\mu)^2 / (2 r \sigma^2)]$', '$a(x) = exp[-(x-\mu)^2 / (2 r \sigma^2)](1 - dexp[-(x-\mu)^2 / (2 r \sigma^2)])$'))

def q_submit(expr):
    global q
    q = eval(expr)
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    axs.lines[3].set_ydata(st.norm(loc=f, scale=s).pdf(xs))
    fig.canvas.draw_idle()
    
def r_submit(expr):
    global r_1
    r_1 = eval(expr)
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    fig.canvas.draw_idle()
    
def r2_submit(expr):
    global r_2
    r_2 = eval(expr)
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    fig.canvas.draw_idle()
    
def d_submit(expr):
    global d
    d = eval(expr)
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    fig.canvas.draw_idle()
    
def hzfunc(label):
    global ax
    ax = ax_dict[label]
    axs.lines[1].set_ydata(synthesise(xs, q, f, s, ax, pi_0))
    fig.canvas.draw_idle()

q_box.on_submit(q_submit)
r_box.on_submit(r_submit)
r2_box.on_submit(r2_submit)
d_box.on_submit(d_submit)
radio.on_clicked(hzfunc)

f_slider = Slider(ax=axf, label='Agent mean', valmin=-3, valmax=3, valinit=0, color="black")
s_slider = Slider(ax=axstd, label="Agent standard deviation", valmin=0, valmax=3, valinit=np.sqrt(0.1), color="black")
mu_slider = Slider(ax=ax_mu, label='Location bias', valmin=-3, valmax=3, valinit=0, color="black")
sig_slider = Slider(ax=ax_sig, label="Scale bias", valmin=0, valmax=3, valinit=1, color="black")

f_slider.on_changed(update); s_slider.on_changed(update); mu_slider.on_changed(update); sig_slider.on_changed(update)
button.on_clicked(reset)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1